## Importações e Configuração Inicial
 - AJUSTAR PARA BUSCAR INFORMAÇÕES DE PONTUAÇÕES MENSAIS

In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json
import time

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-06-02 12:14:40,963 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Função para buscar o nome do time a partir do ID
def buscar_nome_time_por_id(id_time):
    try:
        time_data = api.time(time_id=id_time)
        return time_data.info.nome  
    except CartolaFCError as e:
        print(f"Erro ao buscar nome para o ID {id_time}: {e}")
    except Exception as e:
        print(f"Erro inesperado para o ID {id_time}: {e}")
    return None

ids_times = [13951133,18642587,49852616,3708025,44574236,528730,48498051,6148913,
             3447341,117598,13707047,1273719,44810918,24468241,25401606,18661583,
             29228373,17887202,20696550,212042,479510,4911779,14696986,47543456,
             25565675,25811332,5823700,18421230,18346776,14227419,3015891,186283]

nomes_por_id = {}

# Loop para buscar os nomes
for id_ in ids_times:
    nome = buscar_nome_time_por_id(id_)
    if nome:
        nomes_por_id[id_] = nome
    time.sleep(0.5)  # respeita o rate limit

# Exibe os resultados
for id_, nome in nomes_por_id.items():
    print(f"{id_}: {nome}")


13951133: KING LEONN
18642587: Fedato Futebol Clube
49852616: BORGES CLIMA FUT F.C
3708025: OlhaEleAiF.C!
44574236: Analove10 ITAQUI GRANDE!!
528730: Gremiomaniasm
48498051: Pity10
6148913: E.C. Bororé
3447341: PUXE FC
117598: A Lenda Super Vasco F.c 
13707047: Super Vasco f.c
1273719: Texas Club 2025
44810918: lsauer fc
24468241: Grêmio imortal 37
25401606: mercearia Estrela 
18661583: pura bucha /botafogo
29228373: seralex
17887202: HS SPORTS F.C
20696550: Dom Camillo68
212042: Tatols Beants F.C
479510: TEAM LOPES 99
4911779: MAFRA MARTINS FC
14696986: VASCO MARTINS FC
47543456: Tabajara de Inhaua FC2
25565675: FIGUEIRA DA ILHA
25811332: SERGRILLO
5823700: S.E.R. GRILLO
18421230: Gig@ntte
18346776: KP JUV.
14227419: O clube do povo Itaqui/Rss
3015891: I.B.CASTILHO FC
186283: FBC Colorado


In [3]:
# nomes_por_id 
nomes_times = list(nomes_por_id.values())

# Para copiar e colar no seu código
print("nomes_times = [")
for nome in nomes_times:
    print(f'    "{nome}",')
print("]")

nomes_times = [
    "KING LEONN",
    "Fedato Futebol Clube",
    "BORGES CLIMA FUT F.C",
    "OlhaEleAiF.C!",
    "Analove10 ITAQUI GRANDE!!",
    "Gremiomaniasm",
    "Pity10",
    "E.C. Bororé",
    "PUXE FC",
    "A Lenda Super Vasco F.c ",
    "Super Vasco f.c",
    "Texas Club 2025",
    "lsauer fc",
    "Grêmio imortal 37",
    "mercearia Estrela ",
    "pura bucha /botafogo",
    "seralex",
    "HS SPORTS F.C",
    "Dom Camillo68",
    "Tatols Beants F.C",
    "TEAM LOPES 99",
    "MAFRA MARTINS FC",
    "VASCO MARTINS FC",
    "Tabajara de Inhaua FC2",
    "FIGUEIRA DA ILHA",
    "SERGRILLO",
    "S.E.R. GRILLO",
    "Gig@ntte",
    "KP JUV.",
    "O clube do povo Itaqui/Rss",
    "I.B.CASTILHO FC",
    "FBC Colorado",
]


### Função para buscar o ID de um time pelo nome usando correspondência flexível

In [4]:
# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função para buscar o ID de um time pelo nome usando correspondência flexível
def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)
        nomes_api = [time.nome for time in times]
        nome_proximo = get_close_matches(nome_time, nomes_api, n=1, cutoff=0.6)
        if nome_proximo:
            for time in times:
                if time.nome == nome_proximo[0]:
                    return time.id
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao buscar ID para o time {nome_time}: {e}")
    return None

# Buscar os IDs dos times pelo nome
for nome in nomes_times:
    time_id = buscar_id_time(nome)
    if time_id:
        ids_times[nome] = time_id
    else:
        print(f"Não foi possível encontrar o ID para o time {nome}")

# print(nomes_times)

In [5]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_eliminacao.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls)


✅ Arquivo salvo com sucesso: links_times_cartola_liga_eliminacao.xlsx


,Nome do Time,ID do Time,Link do Time
0,KING LEONN,13951133,https://cartola.globo.com/#!/time/13951133
1,Fedato Futebol Clube,18642587,https://cartola.globo.com/#!/time/18642587
2,BORGES CLIMA FUT F.C,49852616,https://cartola.globo.com/#!/time/49852616
3,OlhaEleAiF.C!,3708025,https://cartola.globo.com/#!/time/3708025
4,Analove10 ITAQUI GRANDE!!,44574236,https://cartola.globo.com/#!/time/44574236
5,Gremiomaniasm,528730,https://cartola.globo.com/#!/time/528730
6,Pity10,48498051,https://cartola.globo.com/#!/time/48498051
7,E.C. Bororé,6148913,https://cartola.globo.com/#!/time/6148913
8,PUXE FC,3447341,https://cartola.globo.com/#!/time/3447341
9,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598


In [6]:
# Obter a rodada atual do mercado
rodada_atual = api.mercado().rodada_atual

print("Rodada Atual:", rodada_atual)

Rodada Atual: 12


## Obter Pontuações por Rodada

In [7]:
# rodada_atual = api.mercado().rodada_atual

# # Verifica se o campeonato começou
# def campeonato_comecou(ids_times):
#     for time_id in ids_times.values():
#         try:
#             pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
#             if pontuacao is not None:
#                 return True
#         except CartolaFCError:
#             continue
#     return False

# # Se campeonato não começou, cria tabela vazia
# if not campeonato_comecou(ids_times):
#     print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
#     total_rodadas = 38
#     df_pontuacoes = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
#     df_pontuacoes[:] = None
# else:
#     df_pontuacoes = pd.DataFrame()
#     times_ativos = dict(ids_times)  # cópia inicial

#     for rodada in range(1, rodada_atual):
#         pontuacoes_rodada = {}
#         print(f"🔄 Rodada {rodada} — Times participantes: {len(times_ativos)}")

#         for nome, time_id in times_ativos.items():
#             try:
#                 time_rodada = api.time(time_id=time_id, rodada=rodada)
#                 pontuacoes_rodada[nome] = time_rodada.ultima_pontuacao
#             except CartolaFCError as e:
#                 print(f"Erro ao acessar pontuação da rodada {rodada} para {nome}: {e}")
#                 pontuacoes_rodada[nome] = None
#             time.sleep(0.4)  # evita rate limit

#         # Adiciona pontuação da rodada
#         df_pontuacoes[f'Rodada {rodada}'] = pd.Series(pontuacoes_rodada)

#         # Elimina o pior colocado (se existir)
#         if len(pontuacoes_rodada) > 1:
#             pior_time = pd.Series(pontuacoes_rodada).idxmin()
#             print(f"❌ Eliminado da Rodada {rodada}: {pior_time}")
#             times_ativos.pop(pior_time, None)

# # Salvar em Excel
# excel_path = 'Pontuacoes_Por_Rodada_Liga_Eliminacao.xlsx'
# df_pontuacoes.to_excel(excel_path)
# print(f"✅ Arquivo Excel salvo: {excel_path}")

# # Salvar em JS
# df_dict = df_pontuacoes.fillna("").to_dict(orient='index')
# js_content = f"const pontuacoesPorRodada = {json.dumps(df_dict, indent=2, ensure_ascii=False)};"

# js_path = "Pontuacoes_Por_Rodada_Liga_Eliminacao.js"
# with open(js_path, "w", encoding="utf-8") as f:
#     f.write(js_content)

# print(f"✅ Arquivo JS salvo: {js_path}")

# # Exibir DataFrame final
# display(df_pontuacoes)


In [8]:
rodada_atual = api.mercado().rodada_atual

# Verifica se o campeonato começou
def campeonato_comecou(ids_times):
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True
        except CartolaFCError:
            continue
    return False

# Se campeonato não começou, cria tabela vazia
if not campeonato_comecou(ids_times):
    print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
    total_rodadas = 38
    df_pontuacoes = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
    df_pontuacoes[:] = None
else:
    df_pontuacoes = pd.DataFrame()
    times_ativos = dict(ids_times)  # cópia inicial

    for rodada in range(1, rodada_atual):
        pontuacoes_rodada = {}
        print(f"🔄 Rodada {rodada} — Times participantes: {len(times_ativos)}")

        for nome, time_id in times_ativos.items():
            try:
                time_rodada = api.time(time_id=time_id, rodada=rodada)
                pontuacoes_rodada[nome] = time_rodada.ultima_pontuacao
            except CartolaFCError as e:
                print(f"Erro ao acessar pontuação da rodada {rodada} para {nome}: {e}")
                pontuacoes_rodada[nome] = None
            time.sleep(0.4)  # evita rate limit

        # Adiciona pontuação da rodada
        df_pontuacoes[f'Rodada {rodada}'] = pd.Series(pontuacoes_rodada)

        # Eliminação
        if len(pontuacoes_rodada) > 1 and rodada <= 18:
            rodada_series = pd.Series(pontuacoes_rodada)
            rodada_series = rodada_series.dropna()

            if rodada <= 9:
                # Elimina 1 time
                eliminado = rodada_series.idxmin()
                print(f"❌ Eliminado da Rodada {rodada}: {eliminado}")
                times_ativos.pop(eliminado, None)

            elif rodada <= 18:
                # Elimina 2 times
                eliminados = rodada_series.nsmallest(2).index.tolist()
                for eliminado in eliminados:
                    print(f"❌ Eliminado da Rodada {rodada}: {eliminado}")
                    times_ativos.pop(eliminado, None)

# Salvar em Excel
excel_path = 'Pontuacoes_Por_Rodada_Liga_Eliminacao.xlsx'
df_pontuacoes.to_excel(excel_path)
print(f"✅ Arquivo Excel salvo: {excel_path}")

# Salvar em JS
df_dict = df_pontuacoes.fillna("").to_dict(orient='index')
js_content = f"const pontuacoesPorRodada = {json.dumps(df_dict, indent=2, ensure_ascii=False)};"

js_path = "Pontuacoes_Por_Rodada_Liga_Eliminacao.js"
with open(js_path, "w", encoding="utf-8") as f:
    f.write(js_content)

print(f"✅ Arquivo JS salvo: {js_path}")

# Exibir DataFrame final
display(df_pontuacoes)


🔄 Rodada 1 — Times participantes: 32
❌ Eliminado da Rodada 1: A Lenda Super Vasco F.c 
🔄 Rodada 2 — Times participantes: 31
❌ Eliminado da Rodada 2: mercearia Estrela 
🔄 Rodada 3 — Times participantes: 30
❌ Eliminado da Rodada 3: seralex
🔄 Rodada 4 — Times participantes: 29
❌ Eliminado da Rodada 4: SERGRILLO
🔄 Rodada 5 — Times participantes: 28
❌ Eliminado da Rodada 5: MAFRA MARTINS FC
🔄 Rodada 6 — Times participantes: 27
❌ Eliminado da Rodada 6: VASCO MARTINS FC
🔄 Rodada 7 — Times participantes: 26
❌ Eliminado da Rodada 7: O clube do povo Itaqui/Rss
🔄 Rodada 8 — Times participantes: 25
❌ Eliminado da Rodada 8: S.E.R. GRILLO
🔄 Rodada 9 — Times participantes: 24
❌ Eliminado da Rodada 9: Tabajara de Inhaua FC2
🔄 Rodada 10 — Times participantes: 23
❌ Eliminado da Rodada 10: KING LEONN
❌ Eliminado da Rodada 10: Gremiomaniasm
🔄 Rodada 11 — Times participantes: 21
❌ Eliminado da Rodada 11: KP JUV.
❌ Eliminado da Rodada 11: pura bucha /botafogo
✅ Arquivo Excel salvo: Pontuacoes_Por_Rodada_Lig

,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11
KING LEONN,79.30,93.04,84.70,84.48,72.61,99.56,87.66,98.45,60.42,40.77,NaN
Fedato Futebol Clube,77.58,77.49,75.42,90.16,81.02,92.06,104.31,98.49,62.97,66.07,110.41
BORGES CLIMA FUT F.C,64.09,93.19,62.90,85.16,68.42,106.15,111.86,150.95,95.92,69.27,114.54
OlhaEleAiF.C!,96.19,102.99,54.12,98.27,92.73,75.26,80.14,82.70,102.70,78.52,92.46
Analove10 ITAQUI GRANDE!!,72.59,76.19,67.40,119.31,86.62,104.20,106.49,135.19,74.32,79.87,105.54
Gremiomaniasm,95.23,59.22,95.55,85.67,86.03,78.92,93.74,73.51,64.00,53.37,NaN
Pity10,75.33,71.22,90.20,112.81,71.31,58.30,99.39,68.89,98.20,54.47,82.26
E.C. Bororé,67.99,86.39,76.92,63.82,80.38,103.66,91.54,83.69,67.68,73.57,120.55
PUXE FC,84.69,95.29,86.95,85.01,73.92,88.76,96.11,103.93,77.90,61.47,93.21
A Lenda Super Vasco F.c,59.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Obter Escalação de um  Time em uma determinada Rodada

In [9]:
# Função auxiliar para verificar se a rodada teve pontuação real
def rodada_tem_pontuacao(api, ids_times, rodada):
    checagens = 0
    sem_pontuacao = 0

    for time_id in list(ids_times.values())[:5]:  # checa os 5 primeiros times apenas
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            if not time_rodada.atletas or all(atleta.pontos in [None, 0.0] for atleta in time_rodada.atletas):
                sem_pontuacao += 1
            checagens += 1
            time.sleep(0.1)  # evitar sobrecarga
        except:
            continue

    return sem_pontuacao < checagens / 2  # só considera a rodada se a maioria tiver pontuação

# Função principal para obter a escalação de um time em uma rodada
def obter_escalacao_time(time_id, rodada):
    try:
        time_rodada = api.time(time_id=time_id, rodada=rodada)

        if not time_rodada.atletas:
            print(f"⚠️ Nenhum jogador encontrado para o time {time_id} na rodada {rodada}.")
            return []

        if all(atleta.pontos in [None, 0.0] for atleta in time_rodada.atletas):
            print(f"⏳ A rodada {rodada} ainda não ocorreu ou não teve pontuação computada.")
            return []

        jogadores = [
            {
                'Nome': atleta.apelido,
                'Posição': atleta.posicao.nome,
                'Pontos': atleta.pontos,
                'Clube': atleta.clube.nome
            }
            for atleta in time_rodada.atletas
        ]
        return jogadores

    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao acessar a escalação da rodada {rodada} para o time {time_id}: {e}")
        return []

# Obter a rodada atual informada pela API
rodada_api = api.mercado().rodada_atual

# Ajustar para considerar apenas rodadas com pontuação finalizada
rodada_confirmada = rodada_api - 1
print(f"📌 A rodada {rodada_api} está em andamento. Coletando dados até a rodada {rodada_confirmada}.")

# Parâmetros
rodada_especifica = 4  # ou qualquer valor que você queira consultar
nome_time = 'SERGRILLO'

# Verificações
if rodada_especifica > rodada_confirmada:
    print(f"⚠️ A Rodada {rodada_especifica} ainda não tem pontuação finalizada. Última rodada com dados válidos é a {rodada_confirmada}.")
elif nome_time not in ids_times:
    print(f"❌ O time '{nome_time}' não foi encontrado no dicionário de IDs.")
else:
    time_cartola = ids_times[nome_time]
    time_escalacao = obter_escalacao_time(time_cartola, rodada_especifica)

    if time_escalacao:
        df_escalacao = pd.DataFrame(time_escalacao)
        df_escalacao.sort_values(by='Posição', inplace=True)
        df_escalacao_pontuacao = df_escalacao['Pontos'].sum()

        print(f'✅ Time do Cartola: {nome_time}')
        print(f'📊 Pontuação na Rodada {rodada_especifica} (sem considerar Capitão): {df_escalacao_pontuacao:.2f}')
        display(df_escalacao.head(14))
    else:
        print("⚠️ Escalação vazia ou rodada ainda não computada.")


📌 A rodada 12 está em andamento. Coletando dados até a rodada 11.
✅ Time do Cartola: SERGRILLO
📊 Pontuação na Rodada 4 (sem considerar Capitão): 48.88


,Nome,Posição,Pontos,Clube
8,Tiquinho Soares,Atacante,2.10,SAN
9,Enner Valencia,Atacante,3.90,INT
10,Yuri Alberto,Atacante,3.50,COR
0,Cássio,Goleiro,6.30,CRU
1,Bernabéi,Lateral,7.10,INT
2,Matheuzinho,Lateral,4.70,COR
5,Marlon Freitas,Meia,1.20,BOT
6,Alan Patrick,Meia,5.50,INT
7,Bruno Henrique,Meia,2.00,INT
11,Rafael Guanaes,Técnico,7.08,MIR


## Obter Escalações Gerais

In [10]:
# Obter a rodada atual informada pela API
rodada_api = api.mercado().rodada_atual
rodada_confirmada = rodada_api - 1
print(f"📌 Rodada atual via API: {rodada_api} | Última rodada com pontuação confirmada: {rodada_confirmada}")

# Função para obter escalações de todos os times em todas as rodadas confirmadas
def obter_escalacoes_gerais(ids_times, rodada_confirmada):
    escalacoes_gerais = []

    for nome_time, time_id in ids_times.items():
        print(f"🔄 Coletando escalações para o time: {nome_time}")
        for rodada in range(1, rodada_confirmada + 1):
            escalação = obter_escalacao_time(time_id, rodada)

            if not escalação:
                print(f"⚠️ Rodada {rodada}: sem escalação disponível para {nome_time}.")
                continue

            for jogador in escalação:
                jogador['Time Cartola'] = nome_time
                jogador['Rodada'] = rodada
                escalacoes_gerais.append(jogador)

            # Evitar sobrecarga de requisições
            time.sleep(0.2)

    return escalacoes_gerais

# Obter escalações gerais até a rodada confirmada
escalacoes_gerais = obter_escalacoes_gerais(ids_times, rodada_confirmada)

# Armazenar em DataFrame
df_escalacoes = pd.DataFrame(escalacoes_gerais)

if df_escalacoes.empty:
    print("⚠️ Nenhuma escalação disponível até o momento. O DataFrame está vazio.")
else:
    # Organizar colunas
    colunas_ordenadas = ['Rodada', 'Time Cartola', 'Nome', 'Posição', 'Clube', 'Pontos']
    df_escalacoes = df_escalacoes[colunas_ordenadas]

    # Salvar em Excel
    df_escalacoes.to_excel('Escalacoes_Gerais_Liga_Eliminacao.xlsx', index=False)
    print("✅ Arquivo 'Escalacoes_Gerais_Liga_Eliminacao.xlsx' salvo com sucesso!")

# -------------------------------------------------------------------------------------------------------------------------------------------#
    # Estrutura para agrupar por rodada
    dados_js = {}

    for rodada, df_rodada in df_escalacoes.groupby('Rodada'):
        dados_js[f'Rodada {rodada}'] = df_rodada.to_dict(orient='records')

    # Converter em string JS
    js_content = f"const escalacoesSerieA = {json.dumps(dados_js, indent=2, ensure_ascii=False)};"

    # Salvar em .js
    js_path = "Escalacoes_Gerais_Liga_Eliminacao.js"
    with open(js_path, "w", encoding="utf-8") as f:
        f.write(js_content)

# -------------------------------------------------------------------------------------------------------------------------------------------#

    # Visualizar primeiras linhas
    display(df_escalacoes.head(32))

📌 Rodada atual via API: 12 | Última rodada com pontuação confirmada: 11
🔄 Coletando escalações para o time: KING LEONN
🔄 Coletando escalações para o time: Fedato Futebol Clube
🔄 Coletando escalações para o time: BORGES CLIMA FUT F.C
🔄 Coletando escalações para o time: OlhaEleAiF.C!
🔄 Coletando escalações para o time: Analove10 ITAQUI GRANDE!!
🔄 Coletando escalações para o time: Gremiomaniasm
🔄 Coletando escalações para o time: Pity10
🔄 Coletando escalações para o time: E.C. Bororé
🔄 Coletando escalações para o time: PUXE FC
🔄 Coletando escalações para o time: A Lenda Super Vasco F.c 
🔄 Coletando escalações para o time: Super Vasco f.c
🔄 Coletando escalações para o time: Texas Club 2025
🔄 Coletando escalações para o time: lsauer fc
🔄 Coletando escalações para o time: Grêmio imortal 37
🔄 Coletando escalações para o time: mercearia Estrela 
🔄 Coletando escalações para o time: pura bucha /botafogo
🔄 Coletando escalações para o time: seralex
🔄 Coletando escalações para o time: HS SPORTS F.C

,Rodada,Time Cartola,Nome,Posição,Clube,Pontos
0,1,KING LEONN,Weverton,Goleiro,PAL,11.50
1,1,KING LEONN,Juninho Capixaba,Lateral,RBB,7.40
2,1,KING LEONN,Piquerez,Lateral,PAL,5.80
3,1,KING LEONN,Alan Franco,Zagueiro,SAO,9.20
4,1,KING LEONN,Arboleda,Zagueiro,SAO,6.60
5,1,KING LEONN,Jadson,Meia,JUV,0.10
6,1,KING LEONN,Matheus Fernandes,Meia,RBB,1.30
7,1,KING LEONN,Hugo Moura,Meia,VAS,3.30
8,1,KING LEONN,Estêvão,Atacante,PAL,2.90
9,1,KING LEONN,Vegetti,Atacante,VAS,14.40


## Função para listar os jogadores mais escalados por posição

In [11]:

# Função para listar os jogadores mais escalados por posição
def listar_jogadores_mais_escalados(df, n_goleiros=3, n_zagueiros=4, n_laterais=4, n_meias=5, n_atacantes=5):
    if df.empty:
        print("⚠️ O DataFrame está vazio. Nenhuma escalação disponível ainda.")
        return pd.DataFrame()

    # Agrupar por nome, posição e clube
    df_agrupado = df.groupby(['Nome', 'Posição', 'Clube'], as_index=False).agg(
        Pontos=('Pontos', 'sum'),
        Quantidade=('Nome', 'size')
    )

    # Filtrar por posição
    goleiros = df_agrupado[df_agrupado['Posição'] == 'Goleiro'].sort_values(by='Quantidade', ascending=False).head(n_goleiros)
    zagueiros = df_agrupado[df_agrupado['Posição'] == 'Zagueiro'].sort_values(by='Quantidade', ascending=False).head(n_zagueiros)
    laterais = df_agrupado[df_agrupado['Posição'] == 'Lateral'].sort_values(by='Quantidade', ascending=False).head(n_laterais)
    meias = df_agrupado[df_agrupado['Posição'] == 'Meia'].sort_values(by='Quantidade', ascending=False).head(n_meias)
    atacantes = df_agrupado[df_agrupado['Posição'] == 'Atacante'].sort_values(by='Quantidade', ascending=False).head(n_atacantes)

    # Concatenar todos os jogadores
    jogadores_mais_escalados = pd.concat([goleiros, zagueiros, laterais, meias, atacantes])

    # colunas = ['Nome', 'Posição', 'Clube', 'Quantidade', 'Pontos']
    colunas = ['Nome', 'Posição', 'Clube', 'Quantidade']
    jogadores_mais_escalados = jogadores_mais_escalados[colunas]

    return jogadores_mais_escalados

# Execução com verificação
jogadores_mais_escalados = listar_jogadores_mais_escalados(df_escalacoes)

if not jogadores_mais_escalados.empty:
    print("📋 Jogadores mais escalados por posição:")
    display(jogadores_mais_escalados.head(10))

    # 🔹 Salvar em Excel
    jogadores_mais_escalados.to_excel("Jogadores_Mais_Escalados_Por_Posicao_Liga_Eliminacao.xlsx", index=False)
    print("✅ Arquivo 'Jogadores_Mais_Escalados_Por_Posicao_Liga_Eliminacao.xlsx' salvo com sucesso.")

    # 🔹 Salvar em JavaScript (.js)
    js_mais_escalados = jogadores_mais_escalados.to_dict(orient="records")
    js_content = f"const jogadoresMaisEscaladosPorPosicao = {json.dumps(js_mais_escalados, indent=2, ensure_ascii=False)};"

    with open("Jogadores_Mais_Escalados_Por_Posicao_Liga_Eliminacao.js", "w", encoding="utf-8") as f:
        f.write(js_content)
    print("✅ Arquivo 'Jogadores_Mais_Escalados_Por_Posicao_Liga_Eliminacao.js' salvo com sucesso.")
else:
    print("⚠️ Nenhum jogador encontrado para salvar.")


📋 Jogadores mais escalados por posição:


,Nome,Posição,Clube,Quantidade
18,Anthoni,Goleiro,INT,52
288,Weverton,Goleiro,PAL,46
255,Ronaldo,Goleiro,BAH,31
192,Léo Ortiz,Zagueiro,FLA,161
228,Pedro Henrique,Zagueiro,RBB,42
139,Ignácio,Zagueiro,FLU,32
162,Junior Alonso,Zagueiro,CAM,31
161,Juninho Capixaba,Lateral,RBB,119
28,Bernabéi,Lateral,INT,79
80,Escobar,Lateral,SAN,52


✅ Arquivo 'Jogadores_Mais_Escalados_Por_Posicao_Liga_Eliminacao.xlsx' salvo com sucesso.
✅ Arquivo 'Jogadores_Mais_Escalados_Por_Posicao_Liga_Eliminacao.js' salvo com sucesso.
